

## Make a questionnaire that asks questions regarding 

-- Use-ases for shortcuts to assign macros to either mouse buttons or keybaord keys



In [44]:
import ollama
import streamlit
import pandas as pd
import numpy as np
import pydantic
from pydantic import BaseModel
from typing import List, Dict, Any
import datetime
import time
import os
from prompt_library import *
import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
PROMPT_LIBRARY["QUESTIONING"].format(system_prompt= "What is your name?")

" You are an AI Assistant asking questions to the user based on the following theme: What is your name?. Adapt the question based on the user's past response. Ask questions using the following format:\n\n    Question: ...\n\n    Options:\n    ...\n    ...\n    ...\n    ...\n    "

## Testing ollama generations

In [42]:
class QuestionSchema(pydantic.BaseModel):
    question: str
    option_1: str
    option_2: str
    option_3: str
    option_4: str

    def __str__(self):
        return f"{self.question} \n A: {self.option_1} \n B: {self.option_2} \n C: {self.option_3} \n D: {self.option_4}"

def ask_question(model, messages):

    response_text = ollama.chat(
        model=model,
        messages=messages,
        format=QuestionSchema.model_json_schema(),
    )['message']['content']

    return QuestionSchema.model_validate_json(response_text)



In [43]:


MODEL = "qwen2.5:3b"
messages = [
    {"role":"system", "content":PROMPT_LIBRARY["QUESTIONING"]}
]

questions = []

while(len(questions) < 5):
    
    output = ask_question(MODEL, messages)
    questions.append(output.question)
    print(output)
    messages.append({"role":"assistant", "content":output.question})
    options = [output.option_1, output.option_2, output.option_3, output.option_4]
    response = random.choice(options)
    print("Answering:", response)
    messages.append({"role":"user", "content":response})

    # time.sleep(1)


Can you tell me more about your favorite hobby or activity that brings you joy? 
 A: I really enjoy hiking in nature 
 B: I love cooking gourmet meals for friends and family 
 C: I prefer playing video games alone 
 D: I like collecting stamps
Answering: I love cooking gourmet meals for friends and family
What type of cuisines do you prefer when cooking gourmet meals? 
 A: Italian 
 B: Japanese 
 C: Indian 
 D: Mexican
Answering: Italian
Which Italian dishes are some of your favorites to cook? 
 A: Pizza 
 B: Lasagna 
 C: Risotto 
 D: Paella
Answering: Pizza
What types of pizza toppings do you like to use in your homemade pizzas? 
 A: Ham and pineapple 
 B: Peppers and onions 
 C: BBQ sauce and beef 
 D: Mushrooms and olives
Answering: Peppers and onions
Are there any specific ingredients or recipes that you find particularly challenging when cooking gourmet meals? 
 A: Yes, I struggle with making authentic Italian pasta sauces. 
 B: I have difficulty finding the right spices for Mexic

In [37]:
print(str(messages))

[{'role': 'system', 'content': " You are an AI Assistant asks random fact based questions to test the knowledge of the user. Based on the user's response, the AI will ask another question. The conversation will continue until the AI has asked 10 questions. At the end of the conversation, the AI will provide a summary of the questions and answers. The user can then choose to start a new conversation or end the session. The AI will ask questions using the following format:\n\n    Question: ...\n\n    Options:\n    ...\n    ...\n    ...\n    ...\n    "}, {'role': 'assistant', 'content': 'What is the largest desert in the world by area?'}, {'role': 'user', 'content': 'Kalahari'}, {'role': 'assistant', 'content': 'What is the largest desert in the world by area?'}, {'role': 'user', 'content': 'Kalahari'}, {'role': 'assistant', 'content': 'What is the largest desert in the world by area?'}, {'role': 'user', 'content': 'Sahara'}, {'role': 'assistant', 'content': 'What is the largest desert in

## Defining the functions to do RAG on for further questioning and recommendations

In [8]:
functions = [
    {"name": "add_numbers", "description": "Adds two numbers and returns the result."},
    {"name": "subtract_numbers", "description": "Subtracts the second number from the first."},
    {"name": "multiply_numbers", "description": "Multiplies two numbers and returns the result."},
    {"name": "divide_numbers", "description": "Divides the first number by the second, returns the quotient."}
]


from sentence_transformers import SentenceTransformer
import numpy as np

# Load pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode function descriptions
function_embeddings = {f["name"]: model.encode(f["description"]) for f in functions}

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\arvin\anaconda3\envs\mnlp-a2\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arvin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

c:\Users\arvin\anaconda3\envs\mnlp-a2\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_function(query):
    query_embedding = model.encode(query)
    similarities = {name: cosine_similarity([query_embedding], [embedding])[0][0]
                    for name, embedding in function_embeddings.items()}
    best_match = max(similarities, key=similarities.get)
    return best_match, similarities[best_match]

def generate_response(query):
    function_name, confidence = retrieve_function(query)
    return f"The most relevant function is '{function_name}'. Confidence: {confidence:.2f}"



query = "What is 4/2?"
response = generate_response(query)
print(response)


The most relevant function is 'subtract_numbers'. Confidence: 0.37
